In [1]:
import numpy as np
import os
import re

In [6]:
# 读取Document-to-Vector的模型
def load_d2v_model(d2v_path):
    from gensim.models.doc2vec import Doc2Vec
    d2v_model = Doc2Vec.load(d2v_path)
    return d2v_model


# 读取神经网络模型
def load_nn_model(nn_path):
    from tensorflow.keras.models import load_model
    nn_model = load_model(nn_path)
    return nn_model


# 读源码文件，转vld文本
def get_vld(codepath):
    command = 'php -dvld.active=1 -dvld.execute=0 -dvld.verbosity=3 ' + str(codepath) + ' 2>&1'
    return os.popen(command).read()
    

# 由vld文本提取出opcode操作码
def get_opcode(vld):
    opcode_list = []
    tokens=re.findall(r'\s(\b[A-Z_]+\b)\s', vld)
    for token in tokens:
        if len(token) != 1:
            opcode_list.append(token)
    return ' '.join(opcode_list)
    

# opcode转文档向量，shape=(256, )
def opcode2vector(opcode_list, d2v_model):
    vector = d2v_model.infer_vector(opcode_list.split(' '))
    vector = np.array(vector)
    print(vector)
    return vector
    

# vector归一化
def std_vector(sample_vector, vec_list):
    from sklearn.preprocessing import StandardScaler
    std_scaler = StandardScaler().fit(vec_list)
    sample_vector_std = std_scaler.transform(sample_vector)
    sample_vector_std = sample_vector_std.reshape(-1, 16, 16, 1)
    return sample_vector_std
    
    
# 读一份代码文件，取出代码文本并进行一系列转换，并进行数据清洗
def transfer_one(code_path):
    d2v_model = load_d2v_model('opcode_doc.d2v')
    vec_list = d2v_model.docvecs.vectors_docs
    vld = get_vld(code_path)
    opcode_list = get_opcode(vld)
    sample_vector = opcode2vector(opcode_list, d2v_model)
    sample_vector_std = std_vector(sample_vector, vec_list)
    return sample_vector_std


# 预测一个样本的情况，返回标签值和预测概率
def predict_one(pred_vec):
#     nn_model = load_nn_model('doc2vec_cov2d.h5')
    nn_model = load_nn_model('doc2vec_lstm.h5')
    pred_label = nn_model.predict(pred_vec)
    pred_proba = nn_model.predict_proba(pred_vec)
    return pred_label, pred_proba

In [7]:
file_path = input('Input code file to be tested > ')
pred_vec = transfer_one(file_path)
label, proba = predict_one(pred_vec)

Input code file to be tested >  .reshape(-1, 1)


php -dvld.active=1 -dvld.execute=0 -dvld.verbosity=3 .reshape(-1, 1) 2>&1
[ 1.9067775e-04  8.4058347e-04  4.0141944e-04  1.7532494e-04
 -2.9822343e-04  5.6989887e-04 -2.4379995e-04  1.5303633e-03
  1.8111827e-03 -4.5530658e-04  1.1395509e-03  1.1287078e-04
  2.6579906e-04  1.6624868e-03 -1.6756403e-03 -1.6127762e-03
 -1.8741469e-03  1.2992963e-03  1.0865498e-03  1.4453599e-03
  1.8696029e-03  1.1685882e-03 -1.5047124e-04  1.0958171e-03
 -1.4911155e-03  5.4656650e-04 -1.3931512e-03  1.7369880e-03
  8.5345004e-05 -3.3335181e-04 -9.1970462e-04  1.0712254e-03
 -1.7128777e-04  2.6732011e-04 -1.8797274e-03  4.5951366e-04
  4.3787391e-04  4.5677341e-04  1.7333910e-03  7.1023556e-04
 -5.4879725e-04 -2.4596893e-04  7.7199686e-04 -1.7178693e-03
  6.5143249e-04  6.6655420e-04 -1.1313182e-03 -1.4495066e-03
 -7.2098302e-04 -5.3237978e-04  2.7420613e-04 -2.3983784e-04
  1.9077103e-03 -1.5545124e-03 -1.1372002e-03 -1.3230097e-03
  5.9807941e-04 -9.6370466e-04 -1.3159854e-04 -9.9833752e-04
 -1.3321501

ValueError: Expected 2D array, got 1D array instead:
array=[ 1.9067775e-04  8.4058347e-04  4.0141944e-04  1.7532494e-04
 -2.9822343e-04  5.6989887e-04 -2.4379995e-04  1.5303633e-03
  1.8111827e-03 -4.5530658e-04  1.1395509e-03  1.1287078e-04
  2.6579906e-04  1.6624868e-03 -1.6756403e-03 -1.6127762e-03
 -1.8741469e-03  1.2992963e-03  1.0865498e-03  1.4453599e-03
  1.8696029e-03  1.1685882e-03 -1.5047124e-04  1.0958171e-03
 -1.4911155e-03  5.4656650e-04 -1.3931512e-03  1.7369880e-03
  8.5345004e-05 -3.3335181e-04 -9.1970462e-04  1.0712254e-03
 -1.7128777e-04  2.6732011e-04 -1.8797274e-03  4.5951366e-04
  4.3787391e-04  4.5677341e-04  1.7333910e-03  7.1023556e-04
 -5.4879725e-04 -2.4596893e-04  7.7199686e-04 -1.7178693e-03
  6.5143249e-04  6.6655420e-04 -1.1313182e-03 -1.4495066e-03
 -7.2098302e-04 -5.3237978e-04  2.7420613e-04 -2.3983784e-04
  1.9077103e-03 -1.5545124e-03 -1.1372002e-03 -1.3230097e-03
  5.9807941e-04 -9.6370466e-04 -1.3159854e-04 -9.9833752e-04
 -1.3321501e-03 -1.5219721